In [1]:
import gc, math, pickle, datetime, os, random
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Memory optimization function
def ds_optimization(dataframe,categorical = []):
    df = dataframe.copy()
    int_types = {
        np.int8 : (np.iinfo(np.int8).min,np.iinfo(np.int8).max),
        np.int16: (np.iinfo(np.int16).min,np.iinfo(np.int16).max),
        np.int32 : (np.iinfo(np.int32).min,np.iinfo(np.int32).max),
        np.int64 : (np.iinfo(np.int64).min,np.iinfo(np.int64).max)
    }
    
    float_types = {
        np.float16: (np.finfo(np.float16).min,np.finfo(np.float16).max), 
        np.float32: (np.finfo(np.float32).min,np.finfo(np.float32).max), 
        np.float64: (np.finfo(np.float64).min,np.finfo(np.float64).max)
    }
    
    for col in df.columns:
        col_type = df[col].dtypes
        col_min = df[col].min()
        col_max = df[col].max()
        if (str(col_type)[:3] == 'int') & (str(col) in categorical):
            df[col] = pd.Categorical(df[col])
            
        elif str(col_type)[:3] == 'int':
            for dtype,drange in int_types.items():
                if (col_min > drange[0]) & (col_max < drange[1]):
                    df[col] = df[col].astype(dtype)
                    break
                    
        elif str(col_type)[:5] == 'float':
            for dtype,drange in float_types.items():
                if (col_min > drange[0]) & (col_max < drange[1]):
                    df[col] = df[col].astype(dtype)
                    break
    return df

In [3]:
# Filter out zeros
def filter_by_columns(df, variables):
    for var in variables:
        df = df[df[var].notnull()]
    return df

In [4]:
# Extract information from datetime object
def extract_year(x):
    return x.year

def extract_month(x):
    return x.month

def extract_day(x):
    return x.day

def extract_hour(x):
    return x.hour

def extract_minute(x):
    return x.minute

def extract_second(x):
    return x.second

In [5]:
def convert_to_int(x):
    try:
        return int(x)
    except:
        return 0

In [6]:
# Load data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
weather_train = pd.read_csv('data/weather_train.csv')
weather_test = pd.read_csv('data/weather_test.csv')
building = pd.read_csv('data/building_metadata.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [7]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [8]:
train.shape

(20216100, 4)

In [9]:
test.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [10]:
test.shape

(41697600, 4)

In [11]:
weather_train.shape, weather_test.shape

((139773, 9), (277243, 9))

In [12]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [13]:
weather_test.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [14]:
building.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [15]:
building.shape

(1449, 6)

In [16]:
submission.head()

,row_id,meter_reading
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [17]:
submission.shape

(24936697, 2)

In [18]:
# Convert data types to save memory
train = ds_optimization(train)
test = ds_optimization(test)
weather_train = ds_optimization(weather_train)
weather_test = ds_optimization(weather_test)
building = ds_optimization(building)

In [19]:
# Joining datasets (train)
df_train = pd.merge(train, building, how='left', on='building_id')
df_train = pd.merge(df_train, weather_train, how='left', on=['site_id', 'timestamp'])

# Joining datasets (test)
df_test = pd.merge(test, building, how='left', on='building_id')
df_test = pd.merge(df_test, weather_test, how='left', on=['site_id', 'timestamp'])

In [20]:
print(df_train['site_id'].unique())
print(df_train['meter'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[0 3 1 2]


In [21]:
df_train.shape, df_test.shape

((20216100, 16), (41697600, 16))

### Handling missing values

In [22]:
df_train.isnull().sum() / df_train.shape[0]

building_id           0.000000
meter                 0.000000
timestamp             0.000000
meter_reading         0.000000
site_id               0.000000
primary_use           0.000000
square_feet           0.000000
year_built            0.599900
floor_count           0.826528
air_temperature       0.004781
cloud_coverage        0.436551
dew_temperature       0.004953
precip_depth_1_hr     0.185447
sea_level_pressure    0.060925
wind_direction        0.071678
wind_speed            0.007107
dtype: float64

In [23]:
df_test.isnull().sum() / df_test.shape[0]

row_id                0.000000
building_id           0.000000
meter                 0.000000
timestamp             0.000000
site_id               0.000000
primary_use           0.000000
square_feet           0.000000
year_built            0.589916
floor_count           0.826050
air_temperature       0.005322
cloud_coverage        0.468664
dew_temperature       0.006255
precip_depth_1_hr     0.187099
sea_level_pressure    0.060359
wind_direction        0.071435
wind_speed            0.007245
dtype: float64

In [24]:
df_train = df_train.drop(['year_built','floor_count', 'cloud_coverage'], axis=1)
df_test = df_test.drop(['year_built', 'floor_count', 'cloud_coverage'], axis=1)

In [25]:
filter_variables = [
    'air_temperature', 'dew_temperature', 'precip_depth_1_hr', 
    'sea_level_pressure', 'wind_direction', 'wind_speed'
]

In [26]:
df_train.shape

(20216100, 13)

In [27]:
df_train['primary_use'].unique()

array(['Education', 'Lodging/residential', 'Office',
       'Entertainment/public assembly', 'Other', 'Retail', 'Parking',
       'Public services', 'Warehouse/storage', 'Food sales and service',
       'Religious worship', 'Healthcare', 'Utility', 'Technology/science',
       'Manufacturing/industrial', 'Services'], dtype=object)

In [29]:
# df_meter_reading_by_usage = describe_stats_by_uses(df_train)

In [30]:
# df_meter_reading_by_usage.head()

In [31]:
beaufort = [
    (0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
    (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)
]

In [34]:
for item in beaufort:
    df_train.loc[(df_train['wind_speed'] >= item[1]) & (df_train['wind_speed'] < item[2]), 'beaufort_scale'] = item[0]
    df_test.loc[(df_test['wind_speed'] >= item[1]) & (df_test['wind_speed'] < item[2]), 'beaufort_scale'] = item[0]

In [40]:
df_train['square_feet'] = np.log(df_train['square_feet'])
df_test['square_feet'] = np.log(df_test['square_feet'])

In [39]:
# df_train.head()

In [38]:
# df_test.head()

In [15]:
# Filter out zeros
# df_train_no_NA = filter_by_columns(df_train, df_train.columns)
# df_test_no_NA = filter_by_columns(df_test, df_test.columns)

In [16]:
# # Convert integer types
# df_train_no_NA['year_built'] = df_train_no_NA['year_built'].apply(convert_to_int)
# df_train_no_NA['floor_count'] = df_train_no_NA['floor_count'].apply(convert_to_int)

# df_test_no_NA['year_built'] = df_test_no_NA['year_built'].apply(convert_to_int)
# df_test_no_NA['floor_count'] = df_test_no_NA['floor_count'].apply(convert_to_int)

In [41]:
# Timestamp cleanup
def timestamp_cleanup(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].apply(extract_year)
    df['month'] = df['timestamp'].apply(extract_month)
    df['day'] = df['timestamp'].apply(extract_day)
    df['hour'] = df['timestamp'].apply(extract_hour)
#     df['minute'] = df['timestamp'].apply(extract_minute)
#     df['second'] = df['timestamp'].apply(extract_second)
    return df

In [42]:
# df_train_no_NA = timestamp_cleanup(df_train_no_NA)
# df_test_no_NA = timestamp_cleanup(df_test_no_NA)

df_train = timestamp_cleanup(df_train)
df_test = timestamp_cleanup(df_test)

### Saving data locally

In [43]:
df_train.to_csv('data/train_processed.csv')

In [44]:
gc.collect()

7

In [ ]:
df_test.to_csv('data/test_processed.csv')

In [ ]:
df_train.shape, df_test.shape, submission.shape

In [ ]:
df_test[:24936697].row_id.unique() == submission.row_id.unique()

In [ ]:
# submission.head()

In [ ]:
# df_train.to_csv('data/train_processed.csv')
# df_test.to_csv('data/test_processed.csv')

### Upload preprocessed data to S3 bucket

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [ ]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()

In [ ]:
# general prefix
prefix='ashrae-energy-prediction'

# *unique* train/test prefixes
train_prefix = '{}/{}'.format(prefix, 'train')
test_prefix = '{}/{}'.format(prefix, 'test')
submission_prefix = '{}/{}'.format(prefix, 'submission')

train_key = 'data/train_processed.csv'
test_key = 'data/test_processed.csv'
submission_key = 'data/sample_submission.csv'

# uploading data to S3, and saving locations
train_path = sagemaker_session.upload_data(train_key, bucket=bucket, key_prefix=train_prefix)
test_path = sagemaker_session.upload_data(test_key, bucket=bucket, key_prefix=test_prefix)
submission_path = sagemaker_session.upload_data(submission_key, bucket=bucket, key_prefix=submission_prefix)

In [ ]:
# check locations
print('Training data is stored in: '+ train_path)
print('Test data is stored in: '+ test_path)
print('Submission data is stored in: '+ submission_path)